In [ ]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS `airports`
OPTIONS (
  location = 'US',
  description = 'US Airport Data'
);

Query is running:   0%|          |

""


In [ ]:
%%bigquery
LOAD DATA INTO `qwiklabs-gcp-01-4b0fd1e23a0f.airports.airport_data`
FROM FILES (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/airports.csv']
)

Query is running:   0%|          |

""


In [2]:
import requests
from typing import Optional, List, Dict
import datetime

# BigQuery and Vertex AI (for Gemini) imports
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel

def generate_weather_alert(
    project_id: str,
    location: str,
    forecast_period: Dict[str, str]
) -> Optional[str]:
    """
    Generates a weather alert using the Gemini API based on a forecast period.

    Args:
        project_id: Your Google Cloud project ID.
        location: The GCP region for your project (e.g., "us-central1").
        forecast_period: A dictionary containing weather data for one period.

    Returns:
        A string containing the generated weather alert, or None on error.
    """
    # Initialize Vertex AI SDK
    vertexai.init(project=project_id, location=location)

    # --- Prompt construction, mirroring your SQL CONCAT logic ---
    prompt = (
        "Create a concise, one-sentence weather alert for an airport based on the "
        "following information:\n"
        f"- Temperature: {forecast_period.get('temperature')}°{forecast_period.get('temperatureUnit')}\n"
        f"- Wind: {forecast_period.get('windSpeed')} from the {forecast_period.get('windDirection')}\n"
        f"- Conditions: {forecast_period.get('shortForecast')}\n"
        f"- Detailed Outlook: {forecast_period.get('detailedForecast')}"
    )

    model = GenerativeModel("gemini-2.0-flash")

    generation_config = {
        "temperature": 0.2,
        "max_output_tokens": 256,
    }

    print("  -> Sending prompt to Gemini...")
    try:
        response = model.generate_content([prompt], generation_config=generation_config)
        return response.candidates[0].content.parts[0].text.strip()
    except Exception as e:
        print(f"  -> Error generating text with Gemini: {e}")
        return None

def get_extended_weather_forecast(lat: float, lon: float) -> Optional[List[Dict[str, str]]]:
    """
    Fetch the extended weather forecast from the U.S. National Weather Service (NWS) API
    based on a given latitude and longitude.
    """
    headers = {
        'User-Agent': 'MyWeatherApp (youremail@example.com)',
        'Accept': 'application/geo+json'
    }

    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    try:
        response = requests.get(points_url, headers=headers, timeout=15)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching points data for {lat},{lon}: {e}")
        return None

    points_data = response.json()
    forecast_url = points_data['properties'].get('forecast')
    if not forecast_url:
        print(f"Forecast URL not found in response for {lat},{lon}.")
        return None

    try:
        forecast_response = requests.get(forecast_url, headers=headers, timeout=15)
        forecast_response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching forecast for {lat},{lon}: {e}")
        return None

    forecast_data = forecast_response.json()
    periods = forecast_data['properties'].get('periods', [])

    if not periods:
        return []

    extended_forecast = []
    for period in periods:
        extended_forecast.append({
            'name': period.get('name'),
            'startTime': period.get('startTime'),
            'temperature': str(period.get('temperature')),
            'temperatureUnit': period.get('temperatureUnit'),
            'windSpeed': period.get('windSpeed'),
            'windDirection': period.get('windDirection'),
            'shortForecast': period.get('shortForecast'),
            'detailedForecast': period.get('detailedForecast')
        })
    return extended_forecast

def load_results_to_bigquery(client: bigquery.Client, project_id: str, dataset_id: str, table_id: str, data: List[Dict]):
    """
    Loads data into a BigQuery table using CREATE OR REPLACE logic.
    """
    if not data:
        print("No data to load to BigQuery.")
        return

    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    print(f"\nAttempting to load {len(data)} records into BigQuery table: {table_ref}")

    # Define the schema for the destination table, including new fields
    schema = [
        bigquery.SchemaField("airport_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("iso_region", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("municipality", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("latitude", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("longitude", "FLOAT64", mode="NULLABLE"),
        bigquery.SchemaField("forecast_period", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("temperature", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("temperature_unit", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("wind_speed", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("wind_direction", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("short_forecast", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("gemini_alert", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("processing_timestamp", "TIMESTAMP", mode="REQUIRED"),
    ]

    # Configure the load job
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        # This disposition creates the table if it doesn't exist,
        # or overwrites it completely if it does.
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )

    try:
        load_job = client.load_table_from_json(data, table_ref, job_config=job_config)
        load_job.result() # Wait for the job to complete
        print(f"Successfully created/replaced table {table_ref} with {load_job.output_rows} rows.")
    except Exception as e:
        print(f"Failed to load data to BigQuery: {e}")

def get_weather_for_airports(project_id: str, location: str, dataset_id: str, source_table_id: str, dest_table_id: str):
    """
    Queries airports, fetches weather, generates alerts, and loads results to BigQuery.
    """
    client = bigquery.Client(project=project_id)
    results_to_load = []

    # Updated query to include new columns and use updated lat/lon names
    query = f"""
        SELECT
            type,
            name as airport_name,
            iso_region,
            municipality,
            latitude_deg as lat,
            longitude_deg as lon
        FROM
            `{project_id}.{dataset_id}.{source_table_id}`
        WHERE type = 'large_airport' AND iso_country = 'US'
    """
    print(f"Running query on BigQuery table: {project_id}.{dataset_id}.{source_table_id}")

    try:
        query_job = client.query(query)
        rows = query_job.result()
    except Exception as e:
        print(f"An error occurred with the BigQuery query: {e}")
        return

    for row in rows:
        airport_name, lat, lon = row.airport_name, row.lat, row.lon
        print("-" * 50)
        print(f"Processing: {airport_name} (Lat: {lat}, Lon: {lon})")

        forecast = get_extended_weather_forecast(lat=lat, lon=lon)

        if forecast:
            current_forecast = forecast[0]
            print(f"  -> Current Weather: {current_forecast['shortForecast']} at {current_forecast['temperature']}°{current_forecast['temperatureUnit']}")

            alert = generate_weather_alert(
                project_id=project_id,
                location=location,
                forecast_period=current_forecast
            )
            if alert:
                print(f"  -> Gemini Alert: {alert}")

            # Prepare the record to be loaded into BigQuery, including new fields
            record = {
                "airport_name": row.airport_name,
                "type": row.type,
                "iso_region": row.iso_region,
                "municipality": row.municipality,
                "latitude": lat,
                "longitude": lon,
                "forecast_period": current_forecast.get("name"),
                "temperature": current_forecast.get("temperature"),
                "temperature_unit": current_forecast.get("temperatureUnit"),
                "wind_speed": current_forecast.get("windSpeed"),
                "wind_direction": current_forecast.get("windDirection"),
                "short_forecast": current_forecast.get("shortForecast"),
                "gemini_alert": alert,
                "processing_timestamp": datetime.datetime.now(datetime.timezone.utc).isoformat(),
            }
            results_to_load.append(record)
        else:
            print("  -> Could not retrieve weather forecast.")

    # After processing all airports, load the collected data into BigQuery
    load_results_to_bigquery(
        client=client,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id=dest_table_id,
        data=results_to_load
    )


if __name__ == "__main__":
    # --- CONFIGURE THESE VALUES ---
    BQ_PROJECT_ID = "qwiklabs-gcp-01-4b0fd1e23a0f"
    GCP_LOCATION = "us-central1"
    BQ_DATASET_ID = "airports"
    BQ_SOURCE_TABLE_ID = "airport_data"
    # This new table will be created or replaced each time the script runs
    BQ_DESTINATION_TABLE_ID = "airport_weather_alerts"

    get_weather_for_airports(
        project_id=BQ_PROJECT_ID,
        location=GCP_LOCATION,
        dataset_id=BQ_DATASET_ID,
        source_table_id=BQ_SOURCE_TABLE_ID,
        dest_table_id=BQ_DESTINATION_TABLE_ID,
    )

Running query on BigQuery table: qwiklabs-gcp-01-4b0fd1e23a0f.airports.airport_data
--------------------------------------------------
Processing: Albuquerque International Sunport (Lat: 35.039976, Lon: -106.608925)
  -> Current Weather: Partly Sunny then Scattered Showers And Thunderstorms at 81°F
  -> Sending prompt to Gemini...
  -> Gemini Alert: **Expect scattered showers and thunderstorms after 3 PM with a high of 81°F and winds from the south at 5-15 mph.**
--------------------------------------------------
Processing: Joint Base Andrews (Lat: 38.810799, Lon: -76.866997)
  -> Current Weather: Sunny at 88°F
  -> Sending prompt to Gemini...
  -> Gemini Alert: **Airport Alert: Sunny skies and moderate southwest winds with gusts up to 22 mph expected; exercise caution during taxi and takeoff.**
--------------------------------------------------
Processing: Hartsfield Jackson Atlanta International Airport (Lat: 33.6367, Lon: -84.428101)
  -> Current Weather: Sunny at 92°F
  -> Sending